In [ ]:
!pip install -U fastapi
!pip install -U uvicorn
!pip install -U tiktoken
!pip install -U openai
!pip install -U langchain-openai
!pip install -U kaleido
!pip install -U python-multipart
!pip install -U cohere
!pip install -U langchain
!pip install -U pypdf
!pip install -U chromadb
!pip install -U 'transformers[sentencepiece]'
#!pip install -U google.colab
!pip install -U colab


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Defaulting to user installation because normal site-packages is not writeable


  Using cached tiktoken-0.6.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.8 MB)


  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.5.2
    Uninstalling tiktoken-0.5.2:
      Successfully uninstalled tiktoken-0.5.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.0.5 requires tiktoken<0.6.0,>=0.5.2, but you have tiktoken 0.6.0 which is incompatible.


...

Defaulting to user installation because normal site-packages is not writeable


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Defaulting to user installation because normal site-packages is not writeable


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# 環境変数の準備
import os
import logging

### Google Colab
#from google.colab import userdata
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)


#import sys
#todo logger設定の高度化
#https://kewton.blog/archives/1350
logging.basicConfig(
    filename="app.log",
    #stream=sys.stdout,
    level=logging.INFO, #ERROR, WARNING, INFO, DEBUG
    format="[%(process)d-%(thread)d]-%(asctime)s-[%(filename)s:%(lineno)d]-%(levelname)s-%(message)s",
    force=True)

### Google Colab
##g_secret ='act_openai'
#g_secret ='kazu_act_openai'
##g_secret ='OPENAI_API_KEY'
#logging.info(f'secret = {g_secret}')
#os.environ["OPENAI_API_KEY"] = userdata.get(g_secret)

from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain import PromptTemplate
from langchain.agents import initialize_agent, Tool

import tiktoken
## Deprecated
#from langchain.embeddings.openai import OpenAIEmbeddings
#from langchain_community.embeddings.openai import OpenAIEmbeddings
#from langchain.chat_models import ChatOpenAI


def _input(prompt):
    print(prompt, end='', flush=True)
    #s = sys.stdin.buffer.readline()
    s = input()
#    s = s.decode('utf-8') \
    s = s.replace("\r", "") \
         .replace("\n", "")
    return s

# 公式のExampleを参考にしたプロンプト組み立て関数
def make_prompt(log):
    prompt = [
        f"{uttr['speaker']}: {uttr['text']}"
        for uttr in log
    ]
    prompt = "<NL>".join(prompt)
    return prompt

# 対話については辞書で持っておくようにする
def add_log(log, role, text):
    log.append({
        "speaker": role,
        "text": text
    })

k = 40
#max_length = 128
max_length = 15


# ちょっとずつ結果を出力してくれるジェネレータ。ChatGPTに聞きました、後述
def gradually_generate(model, tokenizer, token_ids, max_length):
    for _ in range(max_length):
        with torch.no_grad():
            outputs = model(token_ids.to(model.device))

        logits = outputs.logits
        indices_to_remove = logits < torch.topk(logits, k)[0][..., -1, None]
        logits[indices_to_remove] = float('-inf')
        probs = torch.nn.functional.softmax(logits[..., -1, :], dim=-1)
        next_token_id = torch.multinomial(probs, num_samples=1)
        token_ids = torch.cat((token_ids, next_token_id), dim=-1)

        output_str = tokenizer.decode(next_token_id[0])

        yield output_str.replace("<NL>", "\n")

        if "</s>" in output_str:
            break


# https://qiita.com/Isaka-code/items/f45a9a8288710aa807d9
# 【2023年12月最新】LangChainを用いてPDFから演習問題を抽出する方法【RAG】
## メインの処理
if __name__ == "__main__":

    #file_path = "hoken1-ch1.txt"
    #file_path = "/content/drive/MyDrive/simple.txt"
    #file_path = "/content/drive/MyDrive/Actuary/eBooks/hoken1-ch1.txt"
    ###file_path = "/content/drive/MyDrive/Actuary/eBooks/hoken1.txt"
    file_path = "/home/kazu/Books/Actuary-ebook/hoken1.txt" ## Ubuntu@lavie

    # 1. PDFを読み込む
    #todo PDF読み込みはPyPDFではエラーになる。javascriptは読み込めた（はず）が、処理がうまくいかなかったので、テキストに変換して処理している。
    #pages = PyPDFLoader(file_path).load()
    pages = TextLoader(file_path).load()

    # 2. ドキュメントをチャンクに分割
    #docs = CharacterTextSplitter(chunk_size=5000, chunk_overlap=0).split_documents(pages)
    docs = CharacterTextSplitter().split_documents(pages)
    logging.info(f"#docs = {len(docs)}") # 抽出したドキュメントの数

    # 3. 埋め込みモデルの初期化
    ### open ai の embeddingがupgrade
    #https://platform.openai.com/docs/guides/embeddings/embedding-models
    #text-embedding-ada-002 -> text-embedding-3-{small,large}

    # models
    #todo 3.5-turboではtoken数が超過するというエラーを回避する必要がある。
    #GPT_MODEL = "gpt-3.5-turbo"
    #GPT_MODEL = "gpt-4-1106-preview"
    GPT_MODEL = "gpt-4-turbo-preview"
    #EMBEDDING_MODEL = 'text-embedding-ada-002'
    EMBEDDING_MODEL = 'text-embedding-3-small'
    #EMBEDDING_MODEL = 'text-embedding-3-large'

    logging.info(f"GPT_MODEL = {GPT_MODEL}")
    logging.info(f"EMBEDDING_MODEL = {EMBEDDING_MODEL}")
    embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL,tiktoken_model_name="cl100k_base")
    #embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
    #embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

    ### todo; 他のEmbeddingも検証してみたい

    # https://qiita.com/akeyhero/items/ce371bfed64399027c23
    # https://huggingface.co/intfloat/multilingual-e5-large
    # https://huggingface.co/Cohere/Cohere-embed-multilingual-v3.0


    # 4. ベクトルストアにドキュメントを格納
    # todo ChromaDB 永続化
    retriever = Chroma.from_documents(docs, embeddings).as_retriever()

    while "[exit]" not in (keyword := _input("\n保険1教科書第1章から、箇所を特定するキーワードを入力してください（例；営業保険料）。終了は[exit]。\n> ")):
      retrieval_query = keyword + "に関わる箇所を抽出してください。"
      question = keyword + "に関わる箇所を1つ選択して100字程度に要約してください。"

      prompt_template_qa = """あなたは親切で優しいアシスタントです。丁寧に、日本語でお答えください！
      もし以下の情報が探している情報に関連していない場合は、そのトピックに関する自身の知識を用いて質問
      に答えてください。

      {context}

      質問: {question}
      回答（日本語）:"""

      prompt_qa = PromptTemplate(
        template=prompt_template_qa,
        input_variables=["context", "question"]
      )
      chain_type_kwargs = {"prompt": prompt_qa}

      logging.info(f"retrieval_query = {retrieval_query}")
      logging.info(f"question = {question}")
      logging.info(f"prompt_qa = {prompt_qa}")
      logging.info(f"chain_type_kwargs = {chain_type_kwargs}")

      # 5. ドキュメントを抽出
      context_docs = retriever.get_relevant_documents(retrieval_query)
      logging.info(f"#context_docs = {len(context_docs)}") # 抽出したドキュメントの数
      encoding = tiktoken.encoding_for_model(GPT_MODEL)
      str_docs=''
      for doc in context_docs:
        str_docs = ' '.join(doc.page_content)
      logging.info(f"#tokens_context_docs = {len(encoding.encode(str_docs))}") # 抽出したドキュメントのtoken数

      ###pythonだとmetadataはsource(ファイル名)のみで詳細箇所が不明なため、以下のログは意味なし
      #for i in range(len(context_docs)):
        #current_doc = context_docs[i] # i[0..len-1] のドキュメント
        #logging.info(f"i = {i}")
        #logging.info(f"metadata = {current_doc.metadata}") # ドキュメントのメタデータ
        #logging.info(current_doc.page_content) # ドキュメントの中身

      # 6. QAチェーンの初期化・エージェント化
      print('\n***エージェントが要約を実行***')
      #https://qiita.com/mashmoeiar11/items/214984400e3452615ea5
      # RetrievalQAの引数やAgent化
      qa =RetrievalQA.from_chain_type(
          llm=ChatOpenAI(model_name=GPT_MODEL),
          chain_type="stuff",
          retriever=retriever,
          chain_type_kwargs=chain_type_kwargs
      )
      tools = [
          Tool(
              name="vec_search",
              func=qa.run,
              description="vector search result with" + file_path
          ),
      ]
      chat_agent = initialize_agent(
        tools,
        llm=ChatOpenAI(model_name=GPT_MODEL),
        agent = "zero-shot-react-description",
        verbose=True,
        system_message="あなたは親切なアシスタントです。日本語で回答してください!",
      )
      result = chat_agent.run(question)
      print(result)



***エージェントが要約を実行***




> Entering new AgentExecutor chain...


Thought: 変額年金についての情報は、指定されたテキストファイル「hoken1.txt」から探す必要があります。まずは、変額年金に関連する内容を見つけることから始めます。
Action: vec_search
Action Input: 変額年金


Observation: 変額年金は、投資の成果に基づいてその受け取り額が変動する年金商品です。具体的には、加入者が支払った保険料を基に、株式や債券などの投資先に資金を運用し、その運用成果に応じて年金額が増減します。運用結果が良ければ受け取り年金額が増える可能性がありますが、逆に運用結果が悪ければ受け取り額が減少するリスクもあります。

変額年金には、最低保証機能を持つものもあります。これは、投資の結果が悪くても、契約時に定められた最低限の年金額は保証されるというものです。これにより、投資リスクをある程度抑えることが可能です。

また、変額年金には様々な種類があります。例えば、途中で引き出しを行えるタイプや、一定期間後に年金を受け取るタイプなど、多様なニーズに応える商品が提供されています。選択する商品によって、運用方針やリスクの度合い、手数料などが異なるため、加入前には詳細な商品説明を受け、自身のリスク許容度や将来設計に合った商品を選ぶことが重要です。

変額年金は、長期的な資産形成や将来の年金受給を目的とした商品であり、加入する際には、投資リスクや運用成果に大きく左右される点を十分に理解し、慎重な判断が求められます。
Thought:

I now know the final answer.
Final Answer: 変額年金は投資成果により受取額が変動する年金商品。加入者の保険料を株式や債券に運用し、運用成果に応じ年金額が増減。最低保証機能があり、投資リスクを抑えることが可能。商品には多様な種類があり、リスク許容度や将来設計に合った選択が重要。投資リスクへの理解と慎重な判断が求められる。

> Finished chain.
変額年金は投資成果により受取額が変動する年金商品。加入者の保険料を株式や債券に運用し、運用成果に応じ年金額が増減。最低保証機能があり、投資リスクを抑えることが可能。商品には多様な種類があり、リスク許容度や将来設計に合った選択が重要。投資リスクへの理解と慎重な判断が求められる。

保険1教科書第1章から、箇所を特定するキーワードを入力してください（例；営業保険料）。終了は[exit]。
> 

20240204 9:47

Mounted at /content/drive

保険1教科書第1章から、箇所を特定するキーワードを入力してください（例；営業保険料）。終了は[exit]。
> 営業保険料

***エージェントが要約を実行***


> Entering new AgentExecutor chain...
To summarize a section about the premium for operational insurance in about 100 characters, I need to search for relevant content in the provided Hoken1.txt document.

Action: vec_search
Action Input: 営業保険料に関わる箇所
Observation: 営業保険料に関連する部分ですが、営業保険料とは、保険会社が保険商品を販売する際に設定する保険料のことで、以下の要素を考慮して決定されます。

1. 計算基礎率（死亡率、利率、事業費率など）の決定
2. 計算基礎率に基づいた保険料の試算
3. 試算保険料が会社の方針や市場競争状況に適合しているかのテスト
4. 各計算基礎率について将来的に現実的と思われるものを設定し、試算保険料に基づき配当率や利益目標のテストを行う
5. 最終的な料率の計算と法的要件や他の種類との調和性をテストする

営業保険料には、予定事業費として以下のような経費が含まれます。

- 新契約費：新規契約の獲得に必要な経費
- 維持費：保険契約の維持に必要な経費
- 集金費：保険料の収納に必要な経費

事業費は、保険商品ごとに異なり、販売経路や保険種類ごとにも事業費の体系が変わることがあります。また、保険金の支払方法によっても事業費は異なるため、それぞれの事業費を適切に分析し、商品ごとに負担すべき事業費を求めることが必要です。

営業保険料を巡る議論では、解約率の変動が保険業務に与える影響、商品の特性が解約を誘引するかどうか、解約率の変動が収益性や健全性に与える影響なども検討されます。

営業保険料の設定は、保険会社のリスク管理と収益性に大きく影響するため、慎重に行われます。また、保険料の設定は法規制や市場の競争条件にも合わせて行われるため、保険会社はこれらの要素を考慮した上で、適切な営業保険料を決定します。
Thought:I have found a section of the text that describes the concept of operational insurance premium and the factors involved in its determination. To create a summary of about 100 characters, I will focus on the core idea presented in the observation.

Thought: I need to condense the information into a succinct summary that captures the essence of operational insurance premiums.
Final Answer: 営業保険料は保険商品販売のための保険会社が設定する料金です。

> Finished chain.
営業保険料は保険商品販売のための保険会社が設定する料金です。

保険1教科書第1章から、箇所を特定するキーワードを入力してください（例；営業保険料）。終了は[exit]。
> [exit]

----------------

[385-134647042584576]-2024-02-04 00:14:35,911-[<ipython-input-3-a5eb14aa3f50>:20]-INFO-secret = kazu_act_openai
[385-134647042584576]-2024-02-04 00:14:41,245-[<ipython-input-3-a5eb14aa3f50>:109]-INFO-#docs = 183
[385-134647042584576]-2024-02-04 00:14:41,245-[<ipython-input-3-a5eb14aa3f50>:124]-INFO-GPT_MODEL = gpt-4-1106-preview
[385-134647042584576]-2024-02-04 00:14:41,245-[<ipython-input-3-a5eb14aa3f50>:125]-INFO-EMBEDDING_MODEL = text-embedding-3-large
[385-134647042584576]-2024-02-04 00:14:42,082-[posthog.py:20]-INFO-Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
[385-134647042584576]-2024-02-04 00:14:42,244-[base.py:277]-WARNING-Warning: model not found. Using cl100k_base encoding.
[385-134647042584576]-2024-02-04 00:14:58,291-[_client.py:1027]-INFO-HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[385-134647042584576]-2024-02-04 00:15:24,725-[<ipython-input-3-a5eb14aa3f50>:157]-INFO-retrieval_query = 営業保険料に関わる箇所を抽出してください。
[385-134647042584576]-2024-02-04 00:15:24,725-[<ipython-input-3-a5eb14aa3f50>:158]-INFO-question = 営業保険料に関わる箇所を1つ選択して100字程度に要約してください。
[385-134647042584576]-2024-02-04 00:15:24,725-[<ipython-input-3-a5eb14aa3f50>:159]-INFO-prompt_qa = input_variables=['context', 'question'] template='あなたは親切で優しいアシスタントです。丁寧に、日本語でお答えください！\n      もし以下の情報が探している情報に関連していない場合は、そのトピックに関する自身の知識を用いて質問\n      に答えてください。\n\n      {context}\n\n      質問: {question}\n      回答（日本語）:'
[385-134647042584576]-2024-02-04 00:15:24,725-[<ipython-input-3-a5eb14aa3f50>:160]-INFO-chain_type_kwargs = {'prompt': PromptTemplate(input_variables=['context', 'question'], template='あなたは親切で優しいアシスタントです。丁寧に、日本語でお答えください！\n      もし以下の情報が探している情報に関連していない場合は、そのトピックに関する自身の知識を用いて質問\n      に答えてください。\n\n      {context}\n\n      質問: {question}\n      回答（日本語）:')}
[385-134647042584576]-2024-02-04 00:15:24,727-[base.py:277]-WARNING-Warning: model not found. Using cl100k_base encoding.
[385-134647042584576]-2024-02-04 00:15:24,926-[_client.py:1027]-INFO-HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[385-134647042584576]-2024-02-04 00:15:24,989-[<ipython-input-3-a5eb14aa3f50>:164]-INFO-#context_docs = 4
[385-134647042584576]-2024-02-04 00:15:25,000-[<ipython-input-3-a5eb14aa3f50>:169]-INFO-#tokens_context_docs = 5398
[385-134647042584576]-2024-02-04 00:15:27,687-[_client.py:1027]-INFO-HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[385-134647042584576]-2024-02-04 00:15:27,692-[base.py:277]-WARNING-Warning: model not found. Using cl100k_base encoding.
[385-134647042584576]-2024-02-04 00:15:27,847-[_client.py:1027]-INFO-HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[385-134647042584576]-2024-02-04 00:15:56,564-[_client.py:1027]-INFO-HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[385-134647042584576]-2024-02-04 00:16:00,991-[_client.py:1027]-INFO-HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[385-134647042584576]-2024-02-04 00:16:45,740-[<ipython-input-4-9e6d633add65>:20]-INFO-secret = kazu_act_openai
[385-134647042584576]-2024-02-04 00:16:47,015-[<ipython-input-4-9e6d633add65>:109]-INFO-#docs = 183
[385-134647042584576]-2024-02-04 00:16:47,015-[<ipython-input-4-9e6d633add65>:124]-INFO-GPT_MODEL = gpt-4-1106-preview
[385-134647042584576]-2024-02-04 00:16:47,015-[<ipython-input-4-9e6d633add65>:125]-INFO-EMBEDDING_MODEL = text-embedding-ada-002
[385-134647042584576]-2024-02-04 00:17:00,569-[_client.py:1027]-INFO-HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[385-134647042584576]-2024-02-04 00:17:53,621-[<ipython-input-5-9e6d633add65>:20]-INFO-secret = kazu_act_openai
[385-134647042584576]-2024-02-04 00:17:54,803-[<ipython-input-5-9e6d633add65>:109]-INFO-#docs = 183
[385-134647042584576]-2024-02-04 00:17:54,803-[<ipython-input-5-9e6d633add65>:124]-INFO-GPT_MODEL = gpt-4-1106-preview
[385-134647042584576]-2024-02-04 00:17:54,803-[<ipython-input-5-9e6d633add65>:125]-INFO-EMBEDDING_MODEL = text-embedding-ada-002
[385-134647042584576]-2024-02-04 00:18:07,934-[_client.py:1027]-INFO-HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[8828-134245136535552]-2024-02-04 00:42:40,045-[<ipython-input-2-9e6d633add65>:20]-INFO-secret = kazu_act_openai
[8828-134245136535552]-2024-02-04 00:42:45,386-[<ipython-input-2-9e6d633add65>:109]-INFO-#docs = 183
[8828-134245136535552]-2024-02-04 00:42:45,386-[<ipython-input-2-9e6d633add65>:124]-INFO-GPT_MODEL = gpt-4-1106-preview
[8828-134245136535552]-2024-02-04 00:42:45,386-[<ipython-input-2-9e6d633add65>:125]-INFO-EMBEDDING_MODEL = text-embedding-ada-002
[8828-134245136535552]-2024-02-04 00:42:46,335-[posthog.py:20]-INFO-Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
[8828-134245136535552]-2024-02-04 00:42:59,694-[_client.py:1027]-INFO-HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[8828-134245136535552]-2024-02-04 00:43:16,782-[<ipython-input-2-9e6d633add65>:157]-INFO-retrieval_query = 営業保険料に関わる箇所を抽出してください。
[8828-134245136535552]-2024-02-04 00:43:16,782-[<ipython-input-2-9e6d633add65>:158]-INFO-question = 営業保険料に関わる箇所を1つ選択して100字程度に要約してください。
[8828-134245136535552]-2024-02-04 00:43:16,782-[<ipython-input-2-9e6d633add65>:159]-INFO-prompt_qa = input_variables=['context', 'question'] template='あなたは親切で優しいアシスタントです。丁寧に、日本語でお答えください！\n      もし以下の情報が探している情報に関連していない場合は、そのトピックに関する自身の知識を用いて質問\n      に答えてください。\n\n      {context}\n\n      質問: {question}\n      回答（日本語）:'
[8828-134245136535552]-2024-02-04 00:43:16,782-[<ipython-input-2-9e6d633add65>:160]-INFO-chain_type_kwargs = {'prompt': PromptTemplate(input_variables=['context', 'question'], template='あなたは親切で優しいアシスタントです。丁寧に、日本語でお答えください！\n      もし以下の情報が探している情報に関連していない場合は、そのトピックに関する自身の知識を用いて質問\n      に答えてください。\n\n      {context}\n\n      質問: {question}\n      回答（日本語）:')}
[8828-134245136535552]-2024-02-04 00:43:16,998-[_client.py:1027]-INFO-HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[8828-134245136535552]-2024-02-04 00:43:17,037-[<ipython-input-2-9e6d633add65>:164]-INFO-#context_docs = 4
[8828-134245136535552]-2024-02-04 00:43:17,043-[<ipython-input-2-9e6d633add65>:169]-INFO-#tokens_context_docs = 6429
[8828-134245136535552]-2024-02-04 00:43:19,635-[_client.py:1027]-INFO-HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[8828-134245136535552]-2024-02-04 00:43:19,777-[_client.py:1027]-INFO-HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[8828-134245136535552]-2024-02-04 00:43:45,334-[_client.py:1027]-INFO-HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[8828-134245136535552]-2024-02-04 00:43:47,910-[_client.py:1027]-INFO-HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[8828-134245136535552]-2024-02-04 00:45:18,256-[<ipython-input-3-48aae7111bf7>:20]-INFO-secret = kazu_act_openai
[8828-134245136535552]-2024-02-04 00:45:19,367-[<ipython-input-3-48aae7111bf7>:109]-INFO-#docs = 183
[8828-134245136535552]-2024-02-04 00:45:19,367-[<ipython-input-3-48aae7111bf7>:124]-INFO-GPT_MODEL = gpt-4-1106-preview
[8828-134245136535552]-2024-02-04 00:45:19,368-[<ipython-input-3-48aae7111bf7>:125]-INFO-EMBEDDING_MODEL = text-embedding-3-small
[8828-134245136535552]-2024-02-04 00:45:19,500-[base.py:277]-WARNING-Warning: model not found. Using cl100k_base encoding.
[8828-134245136535552]-2024-02-04 00:45:33,690-[_client.py:1027]-INFO-HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[8828-134245136535552]-2024-02-04 00:45:51,165-[<ipython-input-3-48aae7111bf7>:157]-INFO-retrieval_query = 営業保険料に関わる箇所を抽出してください。
[8828-134245136535552]-2024-02-04 00:45:51,165-[<ipython-input-3-48aae7111bf7>:158]-INFO-question = 営業保険料に関わる箇所を1つ選択して100字程度に要約してください。
[8828-134245136535552]-2024-02-04 00:45:51,165-[<ipython-input-3-48aae7111bf7>:159]-INFO-prompt_qa = input_variables=['context', 'question'] template='あなたは親切で優しいアシスタントです。丁寧に、日本語でお答えください！\n      もし以下の情報が探している情報に関連していない場合は、そのトピックに関する自身の知識を用いて質問\n      に答えてください。\n\n      {context}\n\n      質問: {question}\n      回答（日本語）:'
[8828-134245136535552]-2024-02-04 00:45:51,165-[<ipython-input-3-48aae7111bf7>:160]-INFO-chain_type_kwargs = {'prompt': PromptTemplate(input_variables=['context', 'question'], template='あなたは親切で優しいアシスタントです。丁寧に、日本語でお答えください！\n      もし以下の情報が探している情報に関連していない場合は、そのトピックに関する自身の知識を用いて質問\n      に答えてください。\n\n      {context}\n\n      質問: {question}\n      回答（日本語）:')}
[8828-134245136535552]-2024-02-04 00:45:51,166-[base.py:277]-WARNING-Warning: model not found. Using cl100k_base encoding.
[8828-134245136535552]-2024-02-04 00:45:51,361-[_client.py:1027]-INFO-HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[8828-134245136535552]-2024-02-04 00:45:51,409-[<ipython-input-3-48aae7111bf7>:164]-INFO-#context_docs = 4
[8828-134245136535552]-2024-02-04 00:45:51,415-[<ipython-input-3-48aae7111bf7>:169]-INFO-#tokens_context_docs = 5949
[8828-134245136535552]-2024-02-04 00:45:55,354-[_client.py:1027]-INFO-HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[8828-134245136535552]-2024-02-04 00:45:55,359-[base.py:277]-WARNING-Warning: model not found. Using cl100k_base encoding.
[8828-134245136535552]-2024-02-04 00:45:55,495-[_client.py:1027]-INFO-HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import sys
import io

# Shift_JISから逃れるための呪文なので気にしないでください
#sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')
#sys.stderr = io.TextIOWrapper(sys.stderr.buffer, encoding='utf-8')

def _input(prompt):
    print(prompt, end='', flush=True)
    #s = sys.stdin.buffer.readline()
    s = input()
#    s = s.decode('utf-8') \
    s = s.replace("\r", "") \
         .replace("\n", "")
    return s

# 公式のExampleを参考にしたプロンプト組み立て関数
def make_prompt(log):
    prompt = [
        f"{uttr['speaker']}: {uttr['text']}"
        for uttr in log
    ]
    prompt = "<NL>".join(prompt)
    return prompt

# 対話については辞書で持っておくようにする
def add_log(log, role, text):
    log.append({
        "speaker": role,
        "text": text
    })

k = 40
#max_length = 128
max_length = 15


# ちょっとずつ結果を出力してくれるジェネレータ。ChatGPTに聞きました、後述
def gradually_generate(model, tokenizer, token_ids, max_length):
    for _ in range(max_length):
        with torch.no_grad():
            outputs = model(token_ids.to(model.device))

        logits = outputs.logits
        indices_to_remove = logits < torch.topk(logits, k)[0][..., -1, None]
        logits[indices_to_remove] = float('-inf')
        probs = torch.nn.functional.softmax(logits[..., -1, :], dim=-1)
        next_token_id = torch.multinomial(probs, num_samples=1)
        token_ids = torch.cat((token_ids, next_token_id), dim=-1)

        output_str = tokenizer.decode(next_token_id[0])

        yield output_str.replace("<NL>", "\n")

        if "</s>" in output_str:
            break

#tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt-neox-3.6b-instruction-sft", use_fast=False)
#model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt-neox-3.6b-instruction-sft")

tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-medium", use_fast=False)
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")

log = []

while "[exit]" not in (user_message := _input("> ")):
    add_log(log, "ユーザー", user_message)

    prompt = (
        make_prompt(log)
        + "<NL>"
        + "システム: "
    )

    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

    output = ""
    for word in gradually_generate(model, tokenizer, token_ids, max_length):
        print(word, end='', flush=True)
        output += word
    print()

    add_log(log, "システム", output)



実行例の保存その２（保険1全体を読み込ませたバージョン）

Mounted at /content/drive

保険1教科書第1章から、箇所を特定するキーワードを入力してください（例；営業保険料）。終了は[exit]。
> 付加保険料

***エージェントが要約を実行***


> Entering new AgentExecutor chain...
To summarize a section about additional premiums, I need to find relevant content in the text file provided.

Action: vec_search
Action Input: 付加保険料に関する箇所
Observation: 付加保険料に関する箇所について、ご質問があればお答えします。付加保険料とは、保険会社が保険契約の運営に必要な経費を賄うために保険料に上乗せする部分のことを指します。その設定に関しては、保険業法施行規則の改正により、事前認可型から事後モニタリング型の監督体制へと変更されました。これにより、保険会社は収支の十分性や公平性を満たすことを前提とし、各自の責任と判断で付加保険料を設定することになりました。また、事業費関連の収支状況を定期的に測定し、主務官庁へモニタリング報告することが義務付けられています。特に新契約時にかかる費用（イニシャル・コスト）の回収状況や、契約維持・管理のために支出する事業費（ランニング・コスト）の充足状況について、販売経路や保険種類ごとに区分して測定し、付加保険料の十分性や公平性が事後的に検証されることになります。

このトピックに関して、もっと具体的な質問があればお知らせください。お手伝いできることがあれば嬉しいです。
Thought:The provided text explains that additional premiums are part of the insurance premium that is added by insurance companies to cover the necessary expenses for managing an insurance contract. The method for setting these additional premiums has been changed from a pre-approval system to a post-monitoring system due to revisions in the Insurance Business Law Enforcement Regulations. Insurance companies are now responsible for setting these fees on their own, ensuring sufficiency and fairness of revenue and expenses. They are also obligated to regularly measure and report their financial status related to business expenses to the competent authority. The adequacy and fairness of the additional premiums are verified post-factum, taking into account the cost of acquiring new contracts (initial costs) and the cost of maintaining and managing contracts (running costs), differentiated by sales channels and types of insurance.
Final Answer: 付加保険料は保険運営経費を賄うため保険料に上乗せされ、保険業法改正により設定が事後モニタリング型に変更された。保険会社は収支公平性を基に自己責任で設定し、事業費収支を定期的に主務官庁に報告する。

> Finished chain.
付加保険料は保険運営経費を賄うため保険料に上乗せされ、保険業法改正により設定が事後モニタリング型に変更された。保険会社は収支公平性を基に自己責任で設定し、事業費収支を定期的に主務官庁に報告する。

保険1教科書第1章から、箇所を特定するキーワードを入力してください（例；営業保険料）。終了は[exit]。
> 変額年金

***エージェントが要約を実行***


> Entering new AgentExecutor chain...
To find a section related to variable annuities (変額年金) within the provided text file, I should utilize the vec_search tool to locate relevant passages.

Action: vec_search
Action Input: 変額年金
Observation: 変額年金とは、加入者が支払う保険料を株式や債券などの投資商品に運用し、その運用成果によって年金額が変動する年金保険の一種です。加入者は自分のリスク許容度や運用方針に応じて、運用資産を選択することができます。投資の成果が良ければ年金額が増加しますが、悪ければ減少するリスクを持ちます。

日本においては、金融商品取引法に基づく適切な説明と情報提供が求められており、変額年金契約を結ぶ際には投資リスクや手数料などについて充分な理解が必要です。また、運用状況に応じて定期的な見直しや資産配分の変更を行うことも可能です。

変額年金は運用成果が保証されていないため、予定利率を設定する従来型の定額年金保険とは異なります。そのため、将来受け取る年金額が予測しにくいという特徴がありますが、一方でインフレリスクに対する保護や資産増加のチャンスもあります。

利用者は、自身の退職後のライフプランや資産状況、市場の動向を考慮しながら、変額年金が適した商品であるかどうかを検討することが重要です。
Thought:The provided excerpt explains variable annuities as a type of pension insurance where the annuity amount fluctuates based on the performance of investments made with the insurance premiums paid by the policyholder. Policyholders can select the assets they wish to invest in according to their risk tolerance and investment strategy. While there is a chance for the annuity amount to increase with good investment performance, there is also a risk of decrease. In Japan, proper explanation and information provision based on the Financial Instruments and Exchange Act are required when entering into a variable annuity contract, and it is necessary to fully understand investment risks and fees. Regular reviews and asset allocation changes are also possible based on the performance of the investments. Unlike fixed annuities with set interest rates, variable annuities do not guarantee investment outcomes, making the future annuity amount unpredictable. However, they offer protection against inflation risks and opportunities for asset growth. It is crucial for users to consider whether variable annuities are suitable for them, taking into account their post-retirement life plan, asset status, and market trends.

Final Answer: 変額年金は投資成果による年金額の変動が特徴で、市場の動向を考慮しリスクを理解した上で選択する必要があります。日本では金融商品取引法に基づく説明と情報提供が必須です。

> Finished chain.
変額年金は投資成果による年金額の変動が特徴で、市場の動向を考慮しリスクを理解した上で選択する必要があります。日本では金融商品取引法に基づく説明と情報提供が必須です。

保険1教科書第1章から、箇所を特定するキーワードを入力してください（例；営業保険料）。終了は[exit]。
> [exit]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
##https://zenn.dev/ohtaman/articles/run_webapp_on_colab
PORT = 8000
PATH = ''

# 検証用に適当なサーバーを立ち上げておく
!nohup python3 -m http.server $PORT > server.log 2>&1 &

from google.colab import output
output.serve_kernel_port_as_window(PORT, path=PATH)

https://qiita.com/Isaka-code/items/f45a9a8288710aa807d9
https://note.com/npaka/n/n0fd7bd3ed27b

In [ ]:
## https://qiita.com/namn1125/items/bc81f12f7bcfb2494c9c

prompt = [
    {
        "speaker": "ユーザー",
        "text": "日本のおすすめの観光地を教えてください。"
    },
    {
        "speaker": "システム",
        "text": "どの地域の観光地が知りたいですか？"
    },
    {
        "speaker": "ユーザー",
        "text": "渋谷の観光地を教えてください。"
    }
]
prompt = [
    f"{uttr['speaker']}: {uttr['text']}"
    for uttr in prompt
]
prompt = "<NL>".join(prompt)
prompt = (
    prompt
    + "<NL>"
    + "システム: "
)
print(prompt)

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-medium", use_fast=False)
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading
# モデル指定
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")

# 筆者環境ではCPUで計算させるので↓は走りません
if torch.cuda.is_available():
   model = model.to("cuda")

token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        do_sample=True,
        max_new_tokens=128,
        temperature=0.7,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

print('output_ids', output_ids)

output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):])

print('output', output)

output = output.replace("<nl>", "\n")
print(output)


実行例の保存その１

Mounted at /content/drive

保険1教科書第1章から、箇所を特定するキーワードを入力してください（例；営業保険料）。終了は[exit]。
> 営業保険料

***要約を実行***


> Entering new AgentExecutor chain...
I need to find a section in the provided text that discusses the business insurance premium and summarize it in about 100 characters.
Action: vec_search
Action Input: 営業保険料に関わる箇所
Observation: 営業保険料に関わる箇所については、以下の内容が記載されています。

1.1 純保険料と営業保険料
ここでは、純保険料が保険期間中における保険事故の発生を仮定した予定死亡率や予定利率を基に計算されたものであり、営業保険料は純保険料以外に保険会社の経営に必要な経費を加えたものであることが説明されています。

1.2 営業保険料決定の際に考慮すべき点
営業保険料の水準を決める際には、十分性、公平性、収益性などの点を考慮する必要があることが述べられています。

1.3 営業保険料率決定の諸要素
死亡率や利率など、営業保険料率を決定する上で重要な要素が説明されています。

1.4 付加保険料
事業費、安全割増と営業利益、付加保険料に対する監督など、付加保険料に関する詳細が記載されています。

1.5 付加保険料方式とその考え方
付加保険料の考え方や方式、解約価格との関係が述べられています。

1.6 営業保険料の計算
年払基準の営業保険料、月払基準の営業保険料、高額割引など、営業保険料の計算方法が記されています。

1.7 米英における営業保険料の計算
アメリカやイギリスにおける営業保険料の計算方法が紹介されています。

1.8 保険料を巡る議論
保険料率の細分化、販売チャネルの多様化と料率設定、生保商品の価格弾力性についての議論が展開されています。

Appendixには期待値原則についてや細分化料率策定の方法などが記載されており、最後に参考文献が挙げられています。

これらの内容は、特に断らない限り、有配当の個人保険の営業保険料に関して記述されたものです。
Thought:I have identified several sections related to business insurance premiums. Now I will choose one and summarize it.
Action: vec_search
Action Input: 1.1 純保険料と営業保険料の要約
Observation: 純保険料とは、保険期間中に発生すると仮定される保険事故（主に死亡）に基づいて計算される保険料で、予定死亡率や予定利率をもとに、予定された保険給付の支払いに必要な金額を計算したものです。ここでの「予定」は単に将来の基礎率を推測したものではなく、実際の死亡率や利率が変わったとしても追加の保険料を徴収しない「保証」された基礎率です。そのため、これらの予定率には安全割増分が含まれており、契約者への配当の資源ともなります。

一方、営業保険料は、保険給付の対価としての純保険料に加えて、保険会社が保険事業を経営するために必要な経費を加えたものです。純保険料以外の部分を付加保険料と呼び、日本では通常これには会社経営に必要な諸経費のみが含まれますが、外国では営業利益や契約者配当の財源を含む場合もあります。
Thought:Now that I have a summary of section 1.1, I will condense it into approximately 100 characters.
Final Answer: 純保険料は予定死亡率に基づく保険料。営業保険料には経費等が加算される。

> Finished chain.
純保険料は予定死亡率に基づく保険料。営業保険料には経費等が加算される。

保険1教科書第1章から、箇所を特定するキーワードを入力してください（例；営業保険料）。終了は[exit]。
> 付加保険料

***要約を実行***


> Entering new AgentExecutor chain...
To answer the question, I need to find a section in the provided text file that discusses additional premiums (付加保険料). I will use vec_search to locate the relevant information.

Action: vec_search
Action Input: 付加保険料に関する情報を探す
Observation: 付加保険料とは、保険料のうちで保険会社の運営経費や利益を賄うために設定される部分であり、純保険料（リスクのみをカバーするための保険料）とは別に計算されます。2月に行われた保険業法施行規則の改正では、付加保険料の部分に大きな変更がありました。

改正前は事前認可型の監督体制でしたが、改正後は事後モニタリング型の監督体制に変更されました。これにより、付加保険料の設定は、収支の十分性や公平性を満たすことを前提に、算出方法書の認可事項ではなく、各保険会社の責任と判断で設定することとされました。さらに、事業費関連の収支状況を定期的に測定し、主務官庁へのモニタリング報告が義務付けられました。

モニタリングでは、新契約時にかかる費用（イニシャルコスト）の回収状況や、契約維持・管理のために支出する事業費（ランニングコスト）の充足状況について、販売経路や保険種類ごとに区分して測定し、これにより付加保険料の十分性や公平性が事後的に検証されます。

付加保険料の考え方としては、十分性を考慮して設定されるべきであり、普遍性や公平性の問題、費用主義と効用主義の考え方をバランスよくミックスさせた方式が望まれます。また、実際の計算方法としては、過去にはP'=P(1+k)+cやα-β-γ方式などが用いられてきましたが、最近ではα-β-γ方式が広く採用されています。これは新契約費と維持費を経費に組み込んで計算される方式です。

以上が、付加保険料に関する基本的な情報です。もし、さらに具体的な情報が必要であれば、お知らせいただければと思います。
Thought:I have found a section that explains additional premiums and contains several details that need to be succinctly summarized within 100 characters.

Final Answer: 付加保険料は保険運営経費や利益に充てる分。改正で設定は各社の責任となり、公平性や十分性を事後検証する。

> Finished chain.
付加保険料は保険運営経費や利益に充てる分。改正で設定は各社の責任となり、公平性や十分性を事後検証する。

保険1教科書第1章から、箇所を特定するキーワードを入力してください（例；営業保険料）。終了は[exit]。
> [exit]